In [1]:
# initialization
from common_primitives.datamart_augment import Hyperparams as hyper_augment, DataMartAugmentPrimitive
from common_primitives.datamart_download import Hyperparams as hyper_download, DataMartDownloadPrimitive
from common_primitives.denormalize import Hyperparams as hyper_denormalize, DenormalizePrimitive
from common_primitives.datamart_augment import Hyperparams as hyper_augment, DataMartAugmentPrimitive
from d3m.container.dataset import Dataset, D3MDatasetLoader
from datamart_isi.entries import Datamart, DatamartSearchResult
import os
# now we combine wikifier inside datamart, no need to use wikifier from outside primitive
# from dsbox.datapreprocessing.cleaner.wikifier import WikifierHyperparams, Wikifier

In [2]:
"""
Same as the codes in /search_example/, we run wikifier first and search first time to get some serach results
"""
a = Datamart(connection_url="http://dsbox02.isi.edu:9001/blazegraph/namespace/datamart4/sparql")

loader = D3MDatasetLoader()
path = "/Users/minazuki/Desktop/studies/master/2018Summer/data/datasets/seed_datasets_data_augmentation/DA_poverty_estimation/TRAIN/dataset_TRAIN/datasetDoc.json"
# path = "/Users/minazuki/Desktop/studies/master/2018Summer/data/datasets/seed_datasets_current/LL1_336_MS_Geolife_transport_mode_prediction/TRAIN/dataset_TRAIN/datasetDoc.json"
json_file = os.path.abspath(path)
all_dataset_uri = 'file://{}'.format(json_file)
all_dataset = loader.load(dataset_uri=all_dataset_uri)

denormalize_hyperparams = hyper_denormalize.defaults()
denormalize_primitive = DenormalizePrimitive(hyperparams = denormalize_hyperparams)
all_dataset = denormalize_primitive.produce(inputs = all_dataset).value
search_result_wikifier = DatamartSearchResult(search_result={}, supplied_data=None, query_json={}, search_type="wikifier")
wikifiered_result = search_result_wikifier.augment(supplied_data=all_dataset)
search_res = a.search_with_data(query=None, supplied_data=wikifiered_result)
s1 = search_res.get_next_page()

No metadata can provide for wikifier augment


Current column: FIPS
The best matching P node is P882
Current column: State
The best matching P node is P5086
Current column: Area
[ERROR] No candidate P nodes found for input column : [Area]
Current column: RUCCode
A columns with all numerical values and useless detected, skipped


In [3]:
"""
This is an exmaple of wikidata's search results, the serialized result is a str
"""
serialized_res = s1[0].serialize()
print(type(serialized_res))
print("--------------")
print(serialized_res)


<class 'str'>
--------------
{"id": "wikidata search on ['P1082', 'P2046', 'P571'] with column FIPS_wikidata", "score": 1, "metadata": {"search_result": {"p_nodes_needed": ["P1082", "P2046", "P571"], "target_q_node_column_name": "FIPS_wikidata"}, "query_json": null, "search_type": "wikidata"}, "augmentation": {"properties": "join", "left_columns": [6], "right_columns": [4]}, "datamart_type": "isi"}


In [4]:
"""
This is an exmaple of datamart general search results, the serialized result is a str
Because general search result's serialized result are very long, here only displayed first 1000 bytes
"""
serialized_res = s1[4].serialize()
print("type = " + str(type(serialized_res)))
print("--------------")
print("length = " + str(len(serialized_res)))
print("--------------")
print(serialized_res[:1000])


type = <class 'str'>
--------------
length = 27208
--------------
{"id": "D1000003", "score": 0.7053783337684788, "metadata": {"search_result": {"score": {"datatype": "http://www.w3.org/2001/XMLSchema#double", "type": "literal", "value": "0.7053783337684788"}, "rank": {"datatype": "http://www.w3.org/2001/XMLSchema#int", "type": "literal", "value": "3"}, "variable": {"type": "uri", "value": "http://www.wikidata.org/entity/statement/D1000003-8e57cb3e-e518-4dbd-8b59-644cb18bf2c3"}, "dataset": {"type": "uri", "value": "http://www.wikidata.org/entity/D1000003"}, "url": {"type": "uri", "value": "https://raw.githubusercontent.com/usc-isi-i2/datamart-userend/master/example_datasets/Unemployment.csv"}, "file_type": {"datatype": "http://www.w3.org/2001/XMLSchema#string", "type": "literal", "value": "csv"}, "title": {"xml:lang": "en", "type": "literal", "value": "Unemployment.csv"}, "keywords": {"datatype": "http://www.w3.org/2001/XMLSchema#string", "type": "literal", "value": "FIPStxt State Area

In [5]:
"""
Different from /search_example/, this time we use augment primitive to do aumgnet
"""
isi_datamart_url = "http://dsbox02.isi.edu:9001/blazegraph/namespace/datamart4/sparql"
hyper_augment_default = hyper_augment.defaults()
hyper_augment_default = hyper_augment_default.replace({"system_identifier":"ISI"})
hyper_aug_1 = hyper_augment_default.replace({"search_result":s1[0].serialize()})
augment_primitive_1 = DataMartAugmentPrimitive(hyperparams=hyper_aug_1)
augment_res_1 = augment_primitive_1.produce(inputs =wikifiered_result).value

Can't calculate the row length for wikidata search results without supplied data


In [6]:
"""
We can then get sample results as /search_example/
"""
augment_res_1['learningData'].head()

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016,FIPS_wikidata,State_wikidata,area_for_FIPS_wikidata,inception_for_FIPS_wikidata,population_for_FIPS_wikidata
1,1,13297,GA,Walton County,1,11385,Q498312,Q1428,nan,1818-01-01T00:00:00Z,85754
2,2,13137,GA,Habersham County,6,6500,Q501096,Q1428,723,1818-12-15T00:00:00Z,43300
3,3,54017,WV,Doddridge County,9,1460,Q494081,Q1371,830,1845-02-04T00:00:00Z,8344
4,4,55055,WI,Jefferson County,4,7618,Q500958,Q1537,1509,1836-01-01T00:00:00Z,84509
5,5,38065,ND,Oliver County,8,191,Q48933,Q1207,1894,1885-01-01T00:00:00Z,1874


In [7]:
"""
Similarly, run 2 more times of augment
"""
hyper_aug_2 = hyper_augment_default.replace({"search_result":s1[1].serialize()})
augment_primitive_2 = DataMartAugmentPrimitive(hyperparams=hyper_aug_2)
augment_res_2 = augment_primitive_2.produce(inputs =augment_res_1).value

hyper_aug_3 = hyper_augment_default.replace({"search_result":s1[3].serialize()})
augment_primitive_3 = DataMartAugmentPrimitive(hyperparams=hyper_aug_3)
augment_res_3 = augment_primitive_3.produce(inputs =augment_res_2).value

Can't calculate the row length for wikidata search results without supplied data
d3m.primitives.data_preprocessing.ensemble_voting.DSBOX: Primitive's Python path does not adhere to d3m.primitives namespace specification. Reason: primitive family segment must match primitive's primitive family.
d3m.primitives.data_preprocessing.ensemble_voting.DSBOX: Primitive's Python path does not adhere to d3m.primitives namespace specification. Reason: must have a known primitive name segment.


Current column: FIPStxt
The best matching P node is P882
Current column: State
The best matching P node is P5086
Current column: Area_Name
The best matching P node is P2390
[WARNING] Coverage of Q nodes is 0.012210394489668128 which is less than threshold 0.7
Current column: Rural-urban_Continuum_Code_2003
A columns with all numerical values and useless detected, skipped
Current column: Urban_Influence_Code_2003
A columns with all numerical values and useless detected, skipped
Current column: Rural-urban_Continuum_Code_2013
A columns with all numerical values and useless detected, skipped
Current column: Urban_Influence_Code_2013
A columns with all numerical values and useless detected, skipped
Current column: POVALL_2017
[WARNING] Coverage of data is 0.1593613024420789 which is less than threshold 0.7
Current column: CI90LBAll_2017
[WARNING] Coverage of data is 0.2016280525986224 which is less than threshold 0.7
Current column: CI90UBALL_2017
[WARNING] Coverage of data is 0.1286787726

In [8]:
# finally, we get same augmented results
augment_res_3['learningData'].head()

,d3mIndex,FIPS,State,Area,RUCCode,POVALL_2016,FIPS_wikidata,State_wikidata,area_for_FIPS_wikidata,inception_for_FIPS_wikidata,...,PCTPOV517_2017,PCTPOVALL_2017,POV017_2017,POV04_2017,POV517_2017,POVALL_2017,Rural-urban_Continuum_Code_2003,Rural-urban_Continuum_Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013
1,2625,13073,GA,Columbia County,2,10512,Q115307,Q1428,797,1790-12-10T00:00:00Z,...,7.4,6.6,nan,nan,nan,nan,2.0,2.0,2.0,2.0
2,1614,13145,GA,Harris County,2,2883,Q486133,Q1428,1225,1827-12-14T00:00:00Z,...,10.5,8.3,827.0,nan,590.0,nan,2.0,2.0,2.0,2.0
3,2172,13193,GA,Macon County,6,3701,Q492021,Q1428,1051,1837-12-14T00:00:00Z,...,36.8,29.6,981.0,nan,688.0,nan,6.0,6.0,6.0,6.0
4,3023,13075,GA,Cook County,6,4268,Q156431,Q1428,604,1918-01-01T00:00:00Z,...,29.9,21.3,nan,nan,959.0,nan,6.0,6.0,6.0,6.0
5,2653,13271,GA,Telfair County,7,3916,Q691614,Q1428,1150,1807-12-10T00:00:00Z,...,36.0,34.6,nan,nan,713.0,nan,7.0,7.0,9.0,11.0
